# **WEF module:** EODs, AMs, and Envelopes

version 9 June 2024

------------------------------------------------------------------------

# Introduction

Freshwater weakly electric fish evolved independently in South & Central America (the Gymnotiformes) and
 in Africa (the Mormyroids), with more than 200 species currently identified in each clade. Gymnotiform and mormyroid fishes generate an electric field (electrogenesis) through a specialized electric organ (**EO**).

The electric organ discharges (**EODs**) are used in social communication and for sensing the animal's environment. The self-generated EOD is detected by an array of electroreceptors on the skin (electroreception). The fish actively senses distortions to their own EOD caused by nearby objects and the EODs of other fish.

![](images/paste-2520623F.png)

Each species of weakly electric fish produces a weak (\<100 mV/cm) electric field with a characteristic EOD waveform.

These waveforms come in two broad classes, wave-type and pulse-type. Wave-type species produce continuous, nearly sinusoidal EODs that range in frequency from less than 100 Hz to more than 2000 Hz (Fig. 1A). Pulse-type species produce short duration pulses, on the order of a few milliseconds to less than 1 millisecond, with interpulse intervals that can range from 10 milliseconds to well over 100 milliseconds (Fig. 1B).

The EOD waveform and frequency are controlled by an 'electromotor pathway' (Fig. 1C). Action potentials generated by the EO cells (electrocytes) are the fundamental building blocks of the EOD. The near-simultaneous action potentials of a thousand or more electrocytes determine EOD waveform characteristics that can be species, sex, and individually distinct. The EOD frequency (EODf) is controlled by the medullary pacemaker nucleus (Pn), the electrogenic motor command center. The Pn directs the activity of the spinal electromotor neurons, which innervate the electrocytes. The Pn receives inputs from the prepacemaker nuclei (PPn), which modulate EOD frequency.

In this NS&B module, we focus on the electrosensory pathway of wave-type species (Fig. 1C). This electrosensory pathway begins with specialized tuberous electroreceptors that sense spatiotemporal modulations of the fish's own EOD and send this information via primary afferents (EAs) to the hindbrain electrosensory lateral line lobe (ELL). The ELL pyramidal neurons (P-Cells) project to the torus semicircularis (Ts), which then projects to the optic tectum (TeO). The ELL P-Cells also project to the nucleus praeeminentialis (nP), which in turn drives feedback to the ELL, either directly or indirectly via the eminentia granularis posterior (EGp). Although there are multiple pathways for electro-sensory-motor integration, the most direct pathway connects the TS to the PPn via the nucleus electrosensorius (nE).

------------------------------------------------------------------------

# Electric communication signals

When weakly electric fish are in close proximity, their electric fields mix, which results in patterns of interference that can be perceived by each of the fish. Fish can extract information about conspecifics from the interference pattern, and often the fish will respond with specialized modulations of their own electric field (e.g., jamming avoidance response, chirps, gradual rises, etc.). Sometimes the patterns of interference will impair the fish's ability to locate objects in its environment such as prey. In these cases, the fish will shift their EOD frequency away from each other to avoid the interference, a behavior known as Jamming Avoidance Response (JAR). You will use the electrosensory stimuli described below mimic different behavioral conditions.

[**This is the point of this tutorial: What are the patterns of interference, and how do we compute and plot them?**]{.underline}

**Section A:** We make 2 sinewaves - **EOD signals for each fish**.

**Section B:** We add those sinewaves together - similar to **two stationary fish**. [dF is the difference in frequencies]

**Section C:** We multiply a carrier wave (e.g. the EOD of one fish) by a sinewave at a dF to generate **AMs**.

**Section D:** What happens when we have more than 2 fish - **several stationary fish**.

**Section E:** A computational way to simulate an **infinite number of stationary fish** via AMs.

**Section F:** What happens when fish move relative to each other? **They generate envelopes.**

## A) Individual fish EOD/sinewave frequencies

Change the frequencies of the individual fish (S1f and S2f) to see how the following signals interact.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks, butter, filtfilt

Fs = 20000  # Sample rate
EndTime = 2  # Length of signals in seconds
windowidth = 0.15  # Width of plot in seconds

S1f = 250  # Frequency in Hz of first fish
S2f = 260  # Frequency in Hz of second fish

tim = np.arange(1/Fs, EndTime, 1/Fs)  # time stamps for EndTime seconds of sampling
S1 = np.sin(tim * S1f * 2 * np.pi)  # Fish #1 - 250 Hz
S2 = np.sin(tim * S2f * 2 * np.pi) * 0.8  # Fish #2 - 260 Hz and 80% of amplitude of fish #1

# Plot sinewaves
plt.figure()
plt.title('Two EOD signals')
plt.plot(tim, S1, 'b', label='S1')
plt.plot(tim, S2, 'm', label='S2')
plt.xlim([0, windowidth])
plt.legend()
plt.show()


## B) Two fish make 'Beats'

When two wave-type fish are close to one another (e.g., during courtship or agonistic situation), their electric fields sum. As happens whenever you add two sinewaves, the ampltiude will change as the two signals go in and out of phase with each other. This change in amplitude is known as an amplitude modulation (AM). In addition, there is a relative change in phase which is known as phase modulation (PM). The rate of the AMs and PMs is equal to the frequency difference between the two sinewaves. The frequency difference is commonly called the dF.

In [ ]:
ss = S1 + S2                # Add the sinewaves together

ass = np.abs(S1 + S2)       # I want to get the peaks, so take the absolute value
LOCS, _  = find_peaks(ass)  # Here we get the indices (LOCS) of the peaks from the find_peaks function

plt.figure()                # And plot!
plt.title("Summed signal and beat - AMs and PMs")
plt.plot(tim, ss, 'k-', label='Summed signal')
plt.plot(tim[LOCS], ass[LOCS], 'r.-', label='Envelope')
plt.xlim([0, windowidth])
plt.legend()
plt.show()

## C) Sinusoidal amplitude modulations (SAMs) are similar to beats

Sinusoidal amplitude modulations (SAM) are meant to mimic the AMs that emerge when two fish are near each other. Instead of adding two sinewaves, we can multiply a single sinewave with a modulatory signal, such as another sinewave. We can imitate the AMs (but not PMs) generated when two fish are near each other by multiplying one fish's EOD with a sinewave equal to the dF. situation where two wave-type fish are close to one another (e.g., during courtship or agonistic situation). As seen in the introductory lecture, each fish will experience a SAM of its own EOD (i.e., a beat). The beat frequency can range from a few Hz to 400 Hz. Each fish's EOD frequency and the sum are illustrated by running the code below.


In [ ]:
samFreq = abs(S1f - S2f)            # Get the difference in frequencies from our original
                                    # pair of frequencies. This will be our SAM frequency.

sam = 0.5 + (np.cos(samFreq * 2 * np.pi * tim) * 0.4) # Make a sinewave at the SAM frequency.
                                                      # 0.4 is the amplitude, and 0.5 is the offset.
                                                      # In this way, our sinewave will multiply with a range of
                                                      # 1 to 0.1.  We could choose any other values! 
                                                      # Typically depth of modulation in weakly electric fish can be quite low.

S1AM = sam * S1                     # Multiply the original signal by the sam sinewave.

fig, (ax1, ax2) = plt.subplots(2, 1)  # A figure with two rows and one column

ax1.set_title('Signal and AM')        # Plot before multiplication
ax1.plot(tim, S1, 'b', label='S1')
ax1.plot(tim, sam, 'm', label='SAM')
ax1.legend()

ax2.set_title('Modulated signal')      # Plot after multiplications
ax2.plot(tim, S1AM, 'k', label='S1AM')
ax2.plot(tim, sam, 'r', label='SAM')
ax2.legend()

fig.tight_layout()
plt.xlim([0, windowidth])
plt.show()

## D) Sum of Sines (more than two fish)

Now imagine that there are more than two fish. The interactions between the various sinewaves of the fish in the group can make for a complex, often seemingly unpredictable signal.

To simulate this situation, we can add the sinewaves of multiple fish at different EOD frequencies - as happens in nature. Or (as we've just learned) we can multiply S1 signal by an AM signal. In this case, the AM signal would be a sum of all the dFs between each of the fish in this artificial group.

Sum of sines signals can be desinged to appear random to the fish as long as they avoid integer common multiples in the dFs. Such a pseudorandom signal is composed of discrete frequencies that enables us to analyze the response with respect to each frequency indpendently.

In [ ]:
otherFreqs = np.array([255.55, 269.21, 292.03])
otherFreqs.sort()

SoSreal = np.sin(tim * S1f * 2 * np.pi)
for freq in otherFreqs:
    SoSreal += np.sin(2 * np.pi * tim * freq)

aSoSreal = np.abs(SoSreal)
LOCS, _ = find_peaks(aSoSreal)

fig, (ax1, ax2) = plt.subplots(2, 1)
ax1.set_title('Sum of Sines signal by adding EODs')
ax1.plot(tim, SoSreal, 'k', linewidth=1)
ax1.plot(tim[LOCS], aSoSreal[LOCS], 'r.-')
ax1.set_xlim([0, 0.2])

# Difference frequencies
SoSmult = np.zeros(len(tim))
dFFreqs = otherFreqs - S1f
dFFreqs = np.append(dFFreqs, np.diff(otherFreqs))

if len(otherFreqs) == 3:
    dFFreqs = np.append(dFFreqs, otherFreqs[0] - otherFreqs[2])
elif len(otherFreqs) == 4:
    dFFreqs = np.append(dFFreqs, [otherFreqs[0] - otherFreqs[2], otherFreqs[0] - otherFreqs[3], otherFreqs[1] - otherFreqs[3]])

for freq in dFFreqs:
    SoSmult += (1 + np.cos(2 * np.pi * tim * freq))

SoSmult -= min(SoSmult)
SoSmult /= max(abs(SoSmult))
SoS1 = SoSmult * np.sin(tim * S1f * 2 * np.pi)

ax2.set_title('SoS AM by multiplication')
ax2.plot(tim, SoS1 / max(SoS1), 'k', linewidth=1)
ax2.plot(tim, SoSmult, 'r', linewidth=2)
ax2.set_xlim([0, windowidth])

fig.tight_layout()
plt.show()